# Código para la lematización, modelado de tópicos y visualización de datos

### Importes Necesarios

In [1]:
#Numpy and Pandas (necesarios para Spacy)
import numpy as np
import pandas as pd

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from gensim.corpora import Dictionary

#NLTK
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

#spacy y stanza
import stanza
import spacy_stanza
nlp = spacy_stanza.load_pipeline("es", disable=['parser','ner'])

#mysql
import pymysql as sql
import pymysql.cursors

#extras
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Sergio\anaconda3\lib\site-packages\h5py\__init__.py:46: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  from ._conv import register_converters as _register_converters
2024-02-10 11:48:08 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-02-10 11:48:11 INFO: Loading these models for language: es (Spanish):
| Processor    | Package         |
----------------------------------
| tokenize     | ancora          |
| mwt          | ancora          |
| pos          | ancora_charlm   |
| lemma        | ancora_nocharlm |
| constituency | combined_charlm |
| depparse     | ancora_charlm   |
| sentiment    | tass2020        |
| ner          | conll02         |

2024-02-10 11:48:11 WARNING: GPU requested, but is not available!
2024-02-10 11:48:11 INFO: Using device: cpu
2024-02-10 11:48:11 INFO: Loading: tokenize
C:\Users\Sergio\anaconda3\lib\site-packages\sympy\core\cache.py:52: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if V(fastcache.__version__) < '0.4.0':
C:\Users\Sergio\anaconda3\lib\site-packages\setuptools\_distutils\version.py:345: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
2024-02-10 11

## Conexión MySQL

In [2]:
#Conectar a la base de datos en MySQL local
conn = sql.connect(
    host="localhost",
    port=3306,
    user="root",
    passwd="Password1234!",
    db="siglodb_2002-2022")
cursor = conn.cursor()
cursorclass = pymysql.cursors.DictCursor
query = "SELECT titular FROM `siglodb_2002-2022`.siglodb2022 WHERE seccion = 'Internacional';"
cursor.execute(query)
data = cursor.fetchall()
cursor.close()
conn.close()

titulares = [row[0] for row in data]


## Preparación de los datos

In [3]:
#Limpia de manera general los textos (puntuación, minúsculas, etc.)
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(titulares)

In [4]:
# Remueve las palabras de uso común
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


stop_words = stopwords.words('spanish')
stop_words.extend(['cada', 'mil', 'tras'])
data_words_nostops = remove_stopwords(data_words)

In [ ]:
# # Reconocimiento de entidades nombradas
# def ner(texts):
#     lista = []
#     for text in texts:
#         ner_doc = nlp(" ".join(text)) 
#         lista.append([{ent.text} for token in ner_doc])
#     return lista

# named_entities = ner(data_words)

# print (named_entities)

In [ ]:
#Lematización de los textos limpios
def lemmatization(texts):
    texts_out = []
    i = 0
    for text in texts:
        doc = nlp(" ".join(text)) 
        texts_out.append([token.lemma_ for token in doc])
        print (i)
        i = i+1
    return texts_out

data_lemmatized = lemmatization(data_words_nostops)

In [6]:
# Genera el modelo y ejecuta la función para la creación de bigramas y trigramas
bigram = gensim.models.Phrases(data_lemmatized, min_count=5, threshold=50)
trigram = gensim.models.Phrases(bigram[data_lemmatized], threshold=50)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

data_words_bigrams = make_bigrams(data_lemmatized)
data_words_trigrams = make_trigrams(data_words_bigrams)

In [7]:
#Remueve términos frecuentes y crea BoW
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_words_trigrams)

texts = data_words_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words =[]
words_missing_in_tfidf =[]
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow


## Modelo LDA

In [8]:
#Genera el modelo de modelado de tópicos
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

## Visualización

In [9]:
#Visualización de los datos
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis


C:\Users\Sergio\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
C:\Users\Sergio\anaconda3\lib\site-packages\sklearn\manifold\_mds.py:298: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.192185  0.369174       1        1  12.135378
0      0.008219 -0.394672       2        1   8.472234
10     0.244086  0.299311       3        1   7.965194
8     -0.224979 -0.298105       4        1   7.799534
13    -0.142705  0.208542       5        1   6.988680
5      0.042460  0.325736       6        1   6.419140
3      0.358567 -0.040862       7        1   6.383191
7     -0.356493 -0.085106       8        1   5.829633
12     0.210798  0.159563       9        1   5.552857
6     -0.305168  0.136451      10        1   5.353749
15    -0.133616 -0.158641      11        1   4.581939
16     0.100754 -0.268976      12        1   4.485372
11    -0.215618 -0.007208      13        1   3.772589
2      0.236251 -0.196313      14        1   3.594997
14     0.151223 -0.080332      15        1   2.619938
18     0.195485  0.032662      16        1   2.294473
19     0.022891  0.099387      17        1   2.007116
17     0.007776 -0.086173      18        1   1.949831
4     -0.029831 -0.013020      19        1   1.186516
9      0.022084 -0.001416      20        1   0.607639, topic_info=                Term        Freq       Total Category  logprob  loglift
104              eua  842.000000  842.000000  Default  30.0000  30.0000
314            rusia  806.000000  806.000000  Default  29.0000  29.0000
98             pedir  535.000000  535.000000  Default  28.0000  28.0000
421          ucrania  678.000000  678.000000  Default  27.0000  27.0000
300            chino  509.000000  509.000000  Default  26.0000  26.0000
...              ...         ...         ...      ...      ...      ...
3403        suicidio    0.029584    1.984294  Topic20  -8.9712   0.8975
1288          varios    0.029585    1.984391  Topic20  -8.9712   0.8975
2595        salvador    0.029585    1.984556  Topic20  -8.9712   0.8975
946           masivo    0.029585    1.984567  Topic20  -8.9712   0.8975
1507  papa_francisco    0.029585    1.984577  Topic20  -8.9712   0.8974

[761 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
44        6  0.737515   abierto
216       7  0.985473  advertir
158      11  0.910251   afirmar
0         4  0.906246  agresion
362       6  0.908077   alertar
...     ...       ...       ...
355       3  0.966126   visitar
15        8  0.959624  vivienda
29        4  0.973203     vuelo
397       5  0.898430     yemen
307       7  0.993575        él

[265 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 11, 9, 14, 6, 4, 8, 13, 7, 16, 17, 12, 3, 15, 19, 20, 18, 5, 10])